# Prospecção de Dados (Data Mining) DI/FCUL - HA3

## Third Home Assignement (MC/DI/FCUL - 2024)

### Fill in the section below

### GROUP:`01` 

* Catherine Prokhorov (62608) - `X` Hours worked on the project
* Guilherme Cepeda (62931) - `X` Hours worked on the project
* Jorge Aleluia (54549) - `X` Hours worked on the project 
* Rómulo Nogueira (56935) - `X` Hours worked on the project


The purpose of this Home Assignment is:

- Find similar items with Local Sensitivity Hashing
- Do Dimensionality Reduction

**NOTE 1: Students are not allowed to add more cells to the notebook**

**NOTE 2: The notebook must be submited fully executed**

**NOTE 3: Name of notebook should be: HA3_GROUP-XX.ipynb (where XX is the group number)**

### 1. Read the Dataset

The dataset correspond to about 99% of the Human Proteome (set of known Human Proteins - about 19,500), coded with specific structural elements. They are presented in a dictionary where the key is the [UniprotID](https://www.uniprot.org/) of the protein and the value is a set of indices of a specific structural characteristic


Students can use one of two datasets, that are not subsets of each other:
- `data_d3.pickle` - smaller set of structural features (2048)
- `data_d4.pickle` - much larger set of structural features (20736) **Note**: This dataset has been Zipped to fit into moodle. Students should unzip it before usage

Select **one** of the datasets and perform all analyses with it.

It may be adviseable the usage of sparse matrices, especially for the `d4` dataset



proteins_ids = docs

set_characteristics = words

In [1]:
### Your code here
import pickle
import sys
from scipy.sparse import dok_array, csr_array, csc_array, bsr_array, lil_array
import numpy as np

f = open("data_d4.pickle", "rb")

# Load the data
data = pickle.load(f)
proteins_ids = set(data.keys())
set_characteristics = list(data.values())
max_characteristics = max([max(x) for x in set_characteristics])
len_characteristics = max_characteristics + 1

# Some informations
print("We have a total of {} human proteins".format(len(proteins_ids)))
print("And a total of {} specific structural characteristics".format(len_characteristics))
print("This would result in a matrix of size {}x{}".format(len_characteristics, len(proteins_ids)))

# Create the matrix
print()

def make_sparse_matrix(set_characteristics: list[set], proteins_ids:set, len_characteristics:int):
    N = len(proteins_ids) 
    M = len_characteristics

    S_Mat = dok_array((N, M), dtype=np.int8)    
    for i in range(N):
        characteristics_idxs = np.array([x for x in set_characteristics[i]])
        S_Mat[i, characteristics_idxs] = 1

    return S_Mat

print()

dok_matrix = make_sparse_matrix(set_characteristics, proteins_ids, len_characteristics)
dok = pickle.dumps(dok_matrix)

print("Dok matrix size (in MB): ", sys.getsizeof(dok)/(1024*1024))

# Yet Dok sparse matrices, if they are pratical for data loading, they can be very inconvenient for any type of matrix processing so they normally need to be converted to another sparse matrix format, such as CSR or CSC.
print()
# Convert to CSR
csr_matrix = csr_array(dok_matrix)
csr = pickle.dumps(csr_matrix)
print("CSR matrix size (in MB): ", sys.getsizeof(csr)/(1024*1024))

We have a total of 19258 human proteins
And a total of 20736 specific structural characteristics
This would result in a matrix of size 20736x19258


Dok matrix size (in MB):  644.1659421920776

CSR matrix size (in MB):  115.15146541595459


### 2. Perform Local Sensitivity Hashing (LSH)

- examine the selected dataset in terms of similarities and select a set of LSH parameters able to capture the most similar proteins
- Comment your results

**BONUS POINTS**: It might be interesting to identify **some** of the candidate pairs in Uniprot, to check if they share some of the same properties (e.g. for [protein P28223](https://www.uniprot.org/uniprotkb/P28223/entry))


In [31]:
### Add support functions here
import time


def MakeBucketsT(TDocs, perms, N, M, B, R, NB):
    Buckets={}
    all_docs=set(range(N))
    for b in range(B):
        SIGS=np.zeros((N, R), dtype="int32")           # initializes line sig
        for r in range(R):
            perm=perms[b*R+r]
            L=all_docs.copy()                         # gets all docs as a set
            i=0 
            while len(L)>0:
                elem=perm[i]                          # get new element  from permutation
                docs_found=TDocs[elem] & L            # get all the docs with a set bit on that elem that are still on the list
                if len(docs_found)>0:                 # if anything was found
                    SIGS[list(docs_found), r]=i       #   set the line sig to the current position from the perm
                    L=L-docs_found                    #   update the current list removing the found docs
                i+=1                                  # update the current position
                if i==M:                              #this is the case that the document is empty 
                    SIGS[list(L), r]=i                # Highly unlikely in a real data set  
                    L={}
                                                      # we have completed the signature for a given band, 
                                                      # now make the hashes for each document
        for d in range(N):
            bucket = hash(tuple(SIGS[d])) % NB
            Buckets.setdefault((b, bucket),set()).add(d)
    return Buckets

def LSHT(Data, B, R, NB=28934501):
    N,M=Data.shape
    #transpose the data set
    DT=Data.T
    DataT=[set(np.where(DT[i]==1)[0]) for i in range(M)]
    P=B*R
    np.random.seed(3)
    #print("Generating %d permutations for %6.3f similarity" %(P, (1/B)**(1/R)))
    perms=[np.random.permutation(M) for i in range(P)]
    buckets=MakeBucketsT(DataT, perms, N, M, B,R, NB)
    return buckets

In [32]:
### Add processing code here

B = 80
R = 8
s = 0.60
P = 1-(1-s**R)**B
print("The probability that documents share at least one band signature if they are %4.2f similar is: %7.4f" %(s, P))
s = 0.70
P = 1-(1-s**R)**B
print("The probability that documents share at least one band signature if they are %4.2f similar is: %7.4f" %(s, P))

res = LSHT(csr_matrix.toarray(), B, R)

The probability that documents share at least one band signature if they are 0.60 similar is:  0.7421
The probability that documents share at least one band signature if they are 0.70 similar is:  0.9913


In [34]:
for b, buck in res:
    if len(res[(b,buck)])>1:
        print("Band", b, "suggests these similar docs:", res[(b,buck)])


Band 0 suggests these similar docs: {1820, 31}
Band 0 suggests these similar docs: {130, 17318, 172, 2159, 12884, 1304, 6138}
Band 0 suggests these similar docs: {14304, 164}
Band 0 suggests these similar docs: {5706, 12595, 10580, 189}
Band 0 suggests these similar docs: {203, 1374}
Band 0 suggests these similar docs: {320, 701, 16713, 3581, 799}
Band 0 suggests these similar docs: {469, 5606}
Band 0 suggests these similar docs: {15002, 547}
Band 0 suggests these similar docs: {15497, 558}
Band 0 suggests these similar docs: {2881, 569, 1943}
Band 0 suggests these similar docs: {6088, 599}
Band 0 suggests these similar docs: {777, 2722}
Band 0 suggests these similar docs: {12128, 825}
Band 0 suggests these similar docs: {16824, 859}
Band 0 suggests these similar docs: {4610, 3299, 5683, 5335, 953, 13918}
Band 0 suggests these similar docs: {16672, 1275}
Band 0 suggests these similar docs: {1362, 11922}
Band 0 suggests these similar docs: {7931, 1381}
Band 0 suggests these similar docs

##### Your short analysis here

### 3. Do dimensionality reduction

Use the techniques discussued in class to make an appropriate dimensional reduction of the selected dataset. It is not necesary to be extensive, **it is better to select one approach and do it well than try a lot of techniques with poor insights and analysis**

It is important to do some sensitivity analysis, relating the dataset size reduction to the loss of information

In [4]:
### Add supporting functions here

In [5]:
### Add processing code here


##### Your short analysis here

### 3. Discuss your findings [to fill on your own]

- Comment your results above
- Discuss how could they be used for the full Uniprot that currently has about [248 Million proteins](https://www.uniprot.org/uniprotkb/statistics)
